In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from data_prep_func import bucket

In [2]:
column_names = [
    'Survived',
    'Pclass', # ticket class (1, 2, 3)
    'Name',
    'Sex',
    'Age', # Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
    'SibSp', # # of siblings / spouses aboard the Titanic
    'Parch', # # of parents / children aboard the Titanic
    'Ticket', # Ticket number
    'Fare', # Passenger fare
    'Cabin', # Cabin number
    'Embarked' # Port of Embarkation
]

In [3]:
used_columns = [
    'Survived',
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'Embarked'
]
target = 'Survived'
sensitive = 'Sex'

# Reading data

In [4]:
train_data = pd.read_csv('/home/luiz/ufpb/mestrado/code/falsb/benchmark/data/titanic/titanic_train.csv', index_col='PassengerId')
test_data = pd.read_csv('/home/luiz/ufpb/mestrado/code/falsb/benchmark/data/titanic/titanic_test.csv', index_col='PassengerId')

In [5]:
data = pd.concat([train_data, test_data])
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
data.tail()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [7]:
print(len(data.index))

1309


### Remove unused columns

In [8]:
data = pd.DataFrame(data, columns=used_columns)

# Removing missing values

In [9]:
data.isna().sum()

Survived    418
Pclass        0
Sex           0
Age         263
SibSp         0
Parch         0
Fare          1
Embarked      2
dtype: int64

In [10]:
data.dropna(subset=['Survived'], inplace=True) 
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [11]:
print(len(data.index))

891


In [12]:
data.dropna(subset=['Embarked'], inplace=True) 
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

In [13]:
print(len(data.index))

889


In [14]:
data['Age'].fillna(-1, inplace=True) # indicate we dont have this data, but dont drop this data points
data.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [15]:
print(len(data.index))

889


# Normalizing continuous data

In [16]:
continous_attr = ['Fare']

# Age

In [17]:
data['Age'].describe()

count    889.000000
mean      23.541249
std       17.834487
min       -1.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [18]:
data['Age'].apply(np.floor).describe()

count    889.000000
mean      23.525309
std       17.835512
min       -1.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [19]:
data['Age'] = data['Age'].apply(np.floor)

In [20]:
data['Age'] = data['Age'].astype(int)

In [21]:
data[data['Age'] == 0]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
79,1.0,2,male,0,0,2,29.0000,S
306,1.0,1,male,0,1,2,151.5500,S
470,1.0,3,female,0,2,1,19.2583,C
645,1.0,3,female,0,2,1,19.2583,C
756,1.0,2,male,0,1,1,14.5000,S
804,1.0,3,male,0,0,1,8.5167,C
832,1.0,2,male,0,1,1,18.7500,S


#### Bucketing age

In [22]:
data['Age'].head()

PassengerId
1    22
2    38
3    26
4    35
5    35
Name: Age, dtype: int64

In [23]:
buckets = [-1, 2, 12, 18, 25 ,35, 45, 55, 65, 75, 80]

labels = ['Age_' + str(bucket) for bucket in buckets]
dict_labels = {i:labels[i] for i in range(len(labels))}
dict_labels

{0: 'Age_-1',
 1: 'Age_2',
 2: 'Age_12',
 3: 'Age_18',
 4: 'Age_25',
 5: 'Age_35',
 6: 'Age_45',
 7: 'Age_55',
 8: 'Age_65',
 9: 'Age_75',
 10: 'Age_80'}

In [24]:
bucket_train = data['Age'].apply(bucket, buckets=buckets).to_frame()

In [25]:
bucket_train = pd.get_dummies(bucket_train['Age'])
bucket_train.head()

,0,1,2,3,4,5,6,7,8,9,10
PassengerId,,,,,,,,,,,
1,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0


In [26]:
bucket_train = bucket_train.rename(columns=dict_labels)
bucket_train.head()

,Age_-1,Age_2,Age_12,Age_18,Age_25,Age_35,Age_45,Age_55,Age_65,Age_75,Age_80
PassengerId,,,,,,,,,,,
1,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0


In [27]:
data = pd.concat([data, bucket_train], axis=1)
data.drop(['Age'], axis=1, inplace=True)
data.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age_-1,Age_2,Age_12,Age_18,Age_25,Age_35,Age_45,Age_55,Age_65,Age_75,Age_80
PassengerId,,,,,,,,,,,,,,,,,,
1,0.0,3,male,1,0,7.2500,S,0,0,0,0,1,0,0,0,0,0,0
2,1.0,1,female,1,0,71.2833,C,0,0,0,0,0,0,1,0,0,0,0
3,1.0,3,female,0,0,7.9250,S,0,0,0,0,0,1,0,0,0,0,0
4,1.0,1,female,1,0,53.1000,S,0,0,0,0,0,1,0,0,0,0,0
5,0.0,3,male,0,0,8.0500,S,0,0,0,0,0,1,0,0,0,0,0


# Binarizing sex and survived

In [28]:
data['Survived'] = data['Survived'].astype(int)

In [29]:
data['Sex'] = pd.get_dummies(data['Sex'])['female']

In [30]:
data.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age_-1,Age_2,Age_12,Age_18,Age_25,Age_35,Age_45,Age_55,Age_65,Age_75,Age_80
PassengerId,,,,,,,,,,,,,,,,,,
1,0,3,0,1,0,7.2500,S,0,0,0,0,1,0,0,0,0,0,0
2,1,1,1,1,0,71.2833,C,0,0,0,0,0,0,1,0,0,0,0
3,1,3,1,0,0,7.9250,S,0,0,0,0,0,1,0,0,0,0,0
4,1,1,1,1,0,53.1000,S,0,0,0,0,0,1,0,0,0,0,0
5,0,3,0,0,0,8.0500,S,0,0,0,0,0,1,0,0,0,0,0


# One hot encoding categorical data

In [31]:
categorical_attr = ['Embarked']

In [32]:
for attr in categorical_attr:
    column_idx = data.columns.get_loc(attr)
    data = pd.concat([data, pd.get_dummies(data[attr], prefix=attr)], axis=1)

In [33]:
data.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age_-1,Age_2,Age_12,...,Age_25,Age_35,Age_45,Age_55,Age_65,Age_75,Age_80,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,0,1,0,7.2500,S,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,1,1,1,0,71.2833,C,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,1,3,1,0,0,7.9250,S,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,1,1,1,1,0,53.1000,S,0,0,0,...,0,1,0,0,0,0,0,0,0,1
5,0,3,0,0,0,8.0500,S,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [34]:
data.drop(categorical_attr, axis=1, inplace=True)

In [35]:
data.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Age_-1,Age_2,Age_12,Age_18,Age_25,Age_35,Age_45,Age_55,Age_65,Age_75,Age_80,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,,,,
1,0,3,0,1,0,7.2500,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,1,1,1,1,0,71.2833,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,1,3,1,0,0,7.9250,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,1,1,1,1,0,53.1000,0,0,0,0,0,1,0,0,0,0,0,0,0,1
5,0,3,0,0,0,8.0500,0,0,0,0,0,1,0,0,0,0,0,0,0,1


# Reordering the columns

In [36]:
columns_order = [
    'Pclass',
    'Fare',
    'Embarked_C',
    'Embarked_Q',
    'Embarked_S',
    'Age_-1',
    'Age_2',
    'Age_12',
    'Age_18',
    'Age_25',
    'Age_35',
    'Age_45',
    'Age_55',
    'Age_65',
    'Age_75',
    'Age_80',
    'SibSp',
    'Parch',
    'Sex',
    'Survived'
]

In [37]:
data.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Age_-1,Age_2,Age_12,Age_18,Age_25,Age_35,Age_45,Age_55,Age_65,Age_75,Age_80,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,,,,
1,0,3,0,1,0,7.2500,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,1,1,1,1,0,71.2833,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,1,3,1,0,0,7.9250,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,1,1,1,1,0,53.1000,0,0,0,0,0,1,0,0,0,0,0,0,0,1
5,0,3,0,0,0,8.0500,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [38]:
data = data[columns_order]
data.head()

,Pclass,Fare,Embarked_C,Embarked_Q,Embarked_S,Age_-1,Age_2,Age_12,Age_18,Age_25,Age_35,Age_45,Age_55,Age_65,Age_75,Age_80,SibSp,Parch,Sex,Survived
PassengerId,,,,,,,,,,,,,,,,,,,,
1,3,7.2500,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
2,1,71.2833,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1
3,3,7.9250,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
4,1,53.1000,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1
5,3,8.0500,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


# Saving data

In [39]:
print(len(data.index))

889


In [40]:
data.to_csv('../data/titanic/post_prep/titanic.csv')